In [14]:
DATA_DIR = '../datasets/vis_3'

In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
from PIL import Image 
import numpy as np
import os
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import pyplot
import matplotlib.patches as mpatches
import shutil
import time
import pickle
import tqdm
from csv import reader
from functools import reduce
import math
from time import sleep
from pathlib import Path
import csv
import itertools
import random
import gc
import statistics
import pandas as pd
import logging
import functools
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import requests
from matplotlib.widgets import Button
from textwrap import wrap
import matplotlib.gridspec as gridspec
from ipywidgets import widgets, interactive, Checkbox, HBox, VBox
from IPython.display import display, clear_output
from ipykernel.pylab.backend_inline import flush_figures
import ipywidgets

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [16]:
IOU = [97.5835650093748, 86.19199699417621, 86.12450283234904, 0.0, \
           0.0, 1.4885496183206106, 0.0, 60.75683251576734, 87.15415226178416, \
           0.0, 40.305072056896876, 17.762943666558122, 0.0, 89.15674722456014, \
           0.0, 0.0, 0.0, 0.0, 0.0]
GT_pixel_area = [35.32562255859375, 4.6245574951171875, 24.376869201660156, \
                 0.25806427001953125, 0.1140594482421875, 0.26397705078125, \
                 0.1556396484375, 0.8699417114257812, 16.11042022705078, \
                 0.07762908935546875, 0.9389877319335938, 1.1585235595703125, \
                 0.03376007080078125, 8.721733093261719, 0.0, 0.0, 0.0, 0.0, 0.0]
PRED_pixel_area = [39.359092712402344, 4.950141906738281, 24.1973876953125, \
                   0.0, 0.0, 0.2567291259765625, 0.0, 0.9202957153320312, \
                   18.48011016845703, 0.0858306884765625, 2.0238876342773438, \
                   0.2246856689453125, 0.0, 9.359359741210938, 0.0, 0.0, 0.0, \
                   0.0, 0.14247894287109375]

In [17]:
input_img = Image.open(f'{DATA_DIR}/input_img.png')
with open(f'{DATA_DIR}/pred_dict.pickle', 'rb') as f:
    pred_dict = pickle.load(f)
with open(f'{DATA_DIR}/gt_dict.pickle', 'rb') as f:
    gt_dict = pickle.load(f)

In [18]:
color_mapping_before_conversion = {
     0 : (  0,  0,  0),
     1 : (  0,  0,  0),
     2 : (  0,  0,  0),
     3 : (  0,  0,  0),
     4 : (  0,  0,  0),
     5 : (111, 74,  0),
     6 : ( 81,  0, 81),
     7 : (128, 64,128),
     8 : (244, 35,232),
     9 : (250,170,160),
    10 : (230,150,140),
    11 : ( 70, 70, 70),
    12 : (102,102,156),
    13 : (190,153,153),
    14 : (180,165,180),
    15 : (150,100,100),
    16 : (150,120, 90),
    17 : (153,153,153),
    18 : (153,153,153),
    19 : (250,170, 30),
    20 : (220,220,  0),
    21 : (107,142, 35),
    22 : (152,251,152),
    23 : ( 70,130,180),
    24 : (220, 20, 60),
    25 : (255,  0,  0),
    26 : (  0,  0,142),
    27 : (  0,  0, 70),
    28 : (  0, 60,100),
    29 : (  0,  0, 90),
    30 : (  0,  0,110),
    31 : (  0, 80,100),
    32 : (  0,  0,230),
    33 : (119, 11, 32),
    -1 : (  0,  0,142)
}

In [19]:
colors_legend = [
     (128, 64,128,255),
     (244, 35,232,255),
     ( 70, 70, 70,255),
     (102,102,156,255),
     (190,153,153,255),
     (153,153,153,255),
     (250,170, 30,255),
     (220,220,  0,255),
     (107,142, 35,255),
     (152,251,152,255),
     ( 70,130,180,255),
     (220, 20, 60,255),
     (255,  0,  0,255),
     (  0,  0,142,255),
     (  0,  0, 70,255),
     (  0, 60,100,255),
     (  0, 80,100,255),
     (  0,  0,230,255),
     (119, 11, 32,255)
]

In [20]:
color_mapping_after_conversion = {
     255 : (  0,  0,  0),
     0 : (128, 64,128),
     1 : (244, 35,232),
     2 : ( 70, 70, 70),
     3 : (102,102,156),
     4 : (190,153,153),
     5 : (153,153,153),
     6 : (250,170, 30),
     7 : (220,220,  0),
     8 : (107,142, 35),
     9 : (152,251,152),
    10 : ( 70,130,180),
    11 : (220, 20, 60),
    12 : (255,  0,  0),
    13 : (  0,  0,142),
    14 : (  0,  0, 70),
    15 : (  0, 60,100),
    16 : (  0, 80,100),
    17 : (  0,  0,230),
    18 : (119, 11, 32)
}

In [21]:
id2color = dict(map(reversed, color_mapping_after_conversion.items()))

In [22]:
pop_up = {
     255 : "background",
     0 : "road",
     1 : "sidewalk",
     2 : "building",
     3 : "wall",
     4 : "fence",
     5 : "pole",
     6 : "tr. light",
     7 : "tr. sign",
     8 : "veg.",
     9 : "terrain",
    10 : "sky",
    11 : "person",
    12 : "rider",
    13 : "car",
    14 : "truck",
    15 : "bus",
    16 : "train",
    17 : "m. cycle",
    18 : "bicycle"
}

In [23]:
id2class_after_conversion = {
     255 : "unlabeled",
     0 : "road",
     1 : "sidewalk",
     2 : "building",
     3 : "wall",
     4 : "fence",
     5 : "pole",
     6 : "tr. light",
     7 : "tr. sign",
     8 : "veg.",
     9 : "terrain",
    10 : "sky",
    11 : "person",
    12 : "rider",
    13 : "car",
    14 : "truck",
    15 : "bus",
    16 : "train",
    17 : "m. cycle",
    18 : "bicycle"
}

In [24]:
class2id_after_conversion = dict(map(reversed, id2class_after_conversion.items()))

In [25]:
class_list = [id2class_after_conversion[cls] for cls in list(range(19))]

In [26]:
def produce_all_class(class_labels, label_axs, pred_axs, axs1, axs2, axs3, ax, fig):
    num_objects = len(class_labels)
    
    mask_pred = np.full((3,512,1024), 0, dtype='float64')
    mask_gt = np.full((3,512,1024), 0, dtype='float64')
    iou_all = []
    gt_pixel_area = []
    pred_pixel_area = []

    for cls_name in class_labels:
        
        cls = class2id_after_conversion[cls_name]
        
        iou_all.append(IOU[cls])
        gt_pixel_area.append(GT_pixel_area[cls])
        pred_pixel_area.append(PRED_pixel_area[cls])
        
        mask_pred += pred_dict[cls]
        mask_gt += gt_dict[cls]
        
    mask_pred[mask_pred > 1] = 1
    mask_gt[mask_gt > 1] = 1
    
    colors = []
    for cls_name in class_labels:
        
        cls = class2id_after_conversion[cls_name]
        
        (r,g,b) = (color_mapping_after_conversion[cls][0], 
                   color_mapping_after_conversion[cls][1], 
                   color_mapping_after_conversion[cls][2])
        colors.append((r/255,g/255,b/255,1.))
    
    
    # plotting the bottom stuffs
    sizes = 30.
    if (num_objects>=15):
        sizes = 10.
    elif (10<=num_objects and num_objects<15):
        sizes = 15.
    elif (5<=num_objects and num_objects<10):
        sizes = 20.
    else:
        sizes = 30.
    
    axs1.set_xlabel('Given Mask Pixel Occupency (%)')
    axs1.set_ylabel('AI Predicted Mask Pixel Occupency (%)')
    axs1.set_title('Pixel Occupency')
    axs2.set_xlabel('Pixel Occupency (%)')
    axs2.set_ylabel('Object IoU')
    axs2.set_title('Given Mask')
    axs3.set_xlabel('Pixel Occupency (%)')
    axs3.set_ylabel('Object IoU')
    axs3.set_title('AI predicted Mask')
    
    xmn, xmx = -4, 100
    if class_labels:
        xmn, xmx = -4, \
                   max(max(gt_pixel_area),max(pred_pixel_area)) + 3
    ymn, ymx = -4, 100.
    
    axs1.set_xlim([xmn, xmx])
    axs2.set_xlim([xmn, xmx])
    axs3.set_xlim([xmn, xmx])
    axs1.set_ylim([xmn, xmx])
    axs2.set_ylim([ymn, ymx])
    axs3.set_ylim([ymn, ymx])
    
    sc1 = axs1.scatter(gt_pixel_area, pred_pixel_area, s=sizes, c=colors)
    axs1.grid(alpha=0.5)
    axs2.scatter(gt_pixel_area, iou_all, s=sizes, c=colors)
    axs2.grid(alpha=0.5)
    axs3.scatter(pred_pixel_area, iou_all, s=sizes, c=colors)
    axs3.grid(alpha=0.5)
    
    
    # plotting the top stuffs
    label_axs.imshow(mask_gt.transpose(1, 2, 0))
    im = pred_axs.imshow(mask_pred.transpose(1, 2, 0))
    
    
    values = np.unique(mask_pred.ravel())    

    colors.append((0.,0.,0.,1.))
    labs = class_labels + ['background']
    patches = [ mpatches.Patch(color=c, label=l) for c, l in zip(colors, labs) ]
    legend = axs3.legend(handles=patches, bbox_to_anchor=(1.4, 0.4), loc='center', facecolor='white', framealpha=0.5)
    
    display_string = "Current Object: "+"All"
    display_string += " "*(30-len(display_string))
    return mask_pred, mask_gt, sc1

In [27]:
def cam_vis(class_labels):
    if 'ALL' in class_labels: class_labels.remove('ALL')
    fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(14,6))
    fig.dpi = 100
    
    fig.patch.set_facecolor('skyblue')
    fig.patch.set_alpha(0.6)
    
    ax[0, 0].imshow(input_img)
    ax[0, 0].set_xlabel('Input Image', fontsize=12)
    ax[0, 1].set_xlabel('Given Mask', fontsize=12)
    ax[0, 2].set_xlabel('AI Predicted Mask', fontsize=12)
    for i in range(3): ax[0, i].set_xticks([])
    for i in range(3): ax[0, i].set_yticks([])

    mask_pred, mask_gt, sc1 = produce_all_class(class_labels, ax[0, 1], ax[0, 2], ax[1, 0], ax[1, 1], ax[1, 2], ax, fig)
    
    # Hover Code 1
    annot_1 = ax[0, 1].annotate("", xy=(0,0), xytext=(5,5), textcoords="offset points", color='white')
    annot_1.set_visible(False)
    def on_hover_1(event):
        if event.inaxes == ax[0, 1]:
            annot_1.xy = (event.xdata, event.ydata)
            x, y = event.xdata, event.ydata
            x, y = int(x), int(y)
            x, y = x, y-512
            object_here = pop_up[id2color[( int(255*mask_gt[0][y][x]), \
                                            int(255*mask_gt[1][y][x]), \
                                            int(255*mask_gt[2][y][x]) )]]
            annot_1.set_text(f"{object_here}")
            annot_1.set_visible(True)
        else:
            annot_1.set_visible(False)

    cid = fig.canvas.mpl_connect('motion_notify_event', on_hover_1)
    
    # Hover Code 2
    annot_2 = ax[0, 2].annotate("", xy=(0,0), xytext=(5,5), textcoords="offset points", color='white')
    annot_2.set_visible(False)
    def on_hover_2(event):
        if event.inaxes == ax[0, 2]:
            annot_2.xy = (event.xdata, event.ydata)
            x, y = event.xdata, event.ydata
            x, y = int(x), int(y)
            x, y = x, y-512
            object_here = pop_up[id2color[( int(255*mask_pred[0][y][x]), \
                                            int(255*mask_pred[1][y][x]), \
                                            int(255*mask_pred[2][y][x]) )]]
            annot_2.set_text(f"{object_here}")
            annot_2.set_visible(True)
        else:
            annot_2.set_visible(False)

    cid = fig.canvas.mpl_connect('motion_notify_event', on_hover_2)

    plt.tight_layout()
    plt.show()
    
    return plt

In [28]:
%matplotlib widget
classes = ['ALL'] + class_list
checkboxes = [Checkbox(description=w, value=True) for w in classes]

boxes = [[], [], [], []]
for i in range(len(checkboxes)):
    boxes[i//5].append(checkboxes[i])

for i in range(len(boxes)):
    boxes[i] = VBox(boxes[i])
box = HBox(boxes)

out = ipywidgets.Output()


all_clicked = False
all_disabled = False

def updatePlot(**kwargs):
    curr_classes = []
    for c in checkboxes:
        v = c.value
        if v:
            curr_classes.append(c.description)
    with out:
        clear_output(wait=True)
        cam_vis(class_labels=curr_classes)
        
def toggle_all(change):
    global all_clicked, all_disabled
    if not all_disabled:
        all_clicked = True
        for cb in checkboxes[1:]:
            cb.value = checkboxes[0].value
        all_clicked = False

        if change.name == 'value':
            updatePlot()

def change_one(b):
    if b.name == 'value' and not all_clicked:
        flg = True
        for cb in checkboxes[1:]:
            if not cb.value:
                flg = False
        global all_disabled
        all_disabled = True
        checkboxes[0].value = flg
        all_disabled = False
        updatePlot()
        

checkboxes[0].observe(toggle_all)
for cb in checkboxes[1:]:
    cb.observe(change_one)


display(box, out)


with out:
    cam_vis(class_labels=class_list)

Output()